In [0]:
ecommerce_all = spark.read.table('workspace.eccomerce.ecommerce_all')
ecommerce_all

DataFrame[event_time: timestamp, event_type: string, product_id: int, category_id: bigint, category_code: string, brand: string, price: double, user_id: int, user_session: string, _rescued_data: string, source_month: string]

### STEP 1: Baseline Query

In [0]:
%sql
SELECT
  brand,
  SUM(price) AS total_revenue
FROM workspace.eccomerce.ecommerce_all
WHERE event_type = 'purchase'
GROUP BY brand
ORDER BY total_revenue DESC
LIMIT 5;

brand,total_revenue
apple,1.762041449799997E8
samsung,7.500406963000278E7
xiaomi,1.5298960329999916E7
null,1.4931753209999967E7
huawei,7042665.809999964


### STEP 2: Analyze Query Plan

In [0]:
%sql
EXPLAIN FORMATTED
SELECT
  brand,
  SUM(price) AS total_revenue
FROM workspace.eccomerce.ecommerce_all
WHERE event_type = 'purchase'
GROUP BY brand;


plan
"== Physical Plan == AdaptiveSparkPlan (10) +- == Initial Plan == ColumnarToRow (9) +- PhotonResultStage (8) +- PhotonGroupingAgg (7) +- PhotonShuffleExchangeSource (6) +- PhotonShuffleMapStage (5) +- PhotonShuffleExchangeSink (4) +- PhotonGroupingAgg (3) +- PhotonProject (2) +- PhotonScan parquet workspace.eccomerce.ecommerce_all (1) (1) PhotonScan parquet workspace.eccomerce.ecommerce_all Output [3]: [event_type#13511, brand#13515, price#13516] DictionaryFilters: [(event_type#13511 = purchase)] Location: PreparedDeltaFileIndex [s3://dbstorage-prod-wqfdj/uc/7b614f14-75e2-4e1b-a2c4-fa394ed7010b/3017e88b-7c38-4cb7-b33f-640df940c3b6/__unitystorage/catalogs/d8c98fa8-acd2-4310-8cb0-448561ab7cc5/tables/4daba616-03ac-48a0-b5d5-f8177334a71e] ReadSchema: struct RequiredDataFilters: [isnotnull(event_type#13511), (event_type#13511 = purchase)] (2) PhotonProject Input [3]: [event_type#13511, brand#13515, price#13516] Arguments: [brand#13515, price#13516] (3) PhotonGroupingAgg Input [2]: [brand#13515, price#13516] Arguments: [brand#13515], [partial_sum(price#13516) AS sum#13532], [sum#13531], [brand#13515, sum#13532], false (4) PhotonShuffleExchangeSink Input [2]: [brand#13515, sum#13532] Arguments: hashpartitioning(brand#13515, 1024) (5) PhotonShuffleMapStage Input [2]: [brand#13515, sum#13532] Arguments: ENSURE_REQUIREMENTS, [id=#8784] (6) PhotonShuffleExchangeSource Input [2]: [brand#13515, sum#13532] (7) PhotonGroupingAgg Input [2]: [brand#13515, sum#13532] Arguments: [brand#13515], [finalmerge_sum(merge sum#13532) AS sum(price)#13521], [sum(price)#13521], [brand#13515, sum(price)#13521 AS total_revenue#13491], true (8) PhotonResultStage Input [2]: [brand#13515, total_revenue#13491] (9) ColumnarToRow Input [2]: [brand#13515, total_revenue#13491] (10) AdaptiveSparkPlan Output [2]: [brand#13515, total_revenue#13491] Arguments: isFinalPlan=false == Photon Explanation == The query is fully supported by Photon. == Optimizer Statistics (table names per statistics state) == missing = partial = full = ecommerce_all"


### STEP 3: Create a Partitioned Table
Why partition?

To reduce the amount of data scanned when filtering.

Good partition column here:

event_type (low cardinality, frequently filtered)

In [0]:
%sql
DROP TABLE IF EXISTS ecommerce_all_partitioned;
CREATE TABLE ecommerce_all_partitioned
USING DELTA
PARTITIONED BY (event_type)
AS
SELECT *
FROM workspace.eccomerce.ecommerce_all;

num_affected_rows,num_inserted_rows


### STEP 4: Verify Partitions

In [0]:
%sql
SHOW PARTITIONS ecommerce_all_partitioned;

event_type
view
cart
purchase


### STEP 5: Re-analyze Query Plan


In [0]:
%sql
EXPLAIN FORMATTED
SELECT
  brand,
  SUM(price) AS total_revenue
FROM ecommerce_all_partitioned
WHERE event_type = 'purchase'
GROUP BY brand;


plan
"== Physical Plan == AdaptiveSparkPlan (6) +- == Initial Plan == ColumnarToRow (5) +- PhotonResultStage (4) +- PhotonGroupingAgg (3) +- PhotonProject (2) +- PhotonScan parquet workspace.default.ecommerce_all_partitioned (1) (1) PhotonScan parquet workspace.default.ecommerce_all_partitioned Output [3]: [brand#17731, price#17732, event_type#17727] Location: PreparedDeltaFileIndex [s3://dbstorage-prod-wqfdj/uc/7b614f14-75e2-4e1b-a2c4-fa394ed7010b/3017e88b-7c38-4cb7-b33f-640df940c3b6/__unitystorage/catalogs/d8c98fa8-acd2-4310-8cb0-448561ab7cc5/tables/ba097c8b-a79d-41d1-a0ed-6987ccf71762] PartitionFilters: [isnotnull(event_type#17727), (event_type#17727 = purchase)] ReadSchema: struct (2) PhotonProject Input [3]: [brand#17731, price#17732, event_type#17727] Arguments: [brand#17731, price#17732] (3) PhotonGroupingAgg Input [2]: [brand#17731, price#17732] Arguments: [brand#17731], [sum(price#17732)], [sum(price)#17737], [brand#17731, sum(price)#17737 AS total_revenue#17707], true (4) PhotonResultStage Input [2]: [brand#17731, total_revenue#17707] (5) ColumnarToRow Input [2]: [brand#17731, total_revenue#17707] (6) AdaptiveSparkPlan Output [2]: [brand#17731, total_revenue#17707] Arguments: isFinalPlan=false == Photon Explanation == The query is fully supported by Photon. == Optimizer Statistics (table names per statistics state) == missing = partial = full = ecommerce_all_partitioned"


### STEP 6: Apply Z-ORDER Optimization
Why Z-ORDER?

Improves performance for frequently filtered or grouped columns.

Good Z-ORDER columns:

brand

product_id

In [0]:
%sql
OPTIMIZE ecommerce_all_partitioned
ZORDER BY (brand, product_id);


path,metrics
,"List(58, 12, List(28073390, 133341178, 4.607299729310345E7, 58, 2672233843), List(133340710, 399837977, 2.6748490183333334E8, 12, 3209818822), 3, List(minCubeSize(107374182400), List(0, 0), List(13, 3269807466), 0, List(12, 3209818822), 2, null), null, 0, 1, 13, 1, false, 0, 0, 1768719857010, 1768719894979, 8, 2, null, List(0, 0), null, 11, 11, 151569, 0, null)"


### STEP 7: Run Optimized Query

In [0]:
%sql
SELECT
  brand,
  SUM(price) AS total_revenue
FROM ecommerce_all_partitioned
WHERE event_type = 'purchase'
GROUP BY brand
limit 5;


brand,total_revenue
apple,1.7620414497995704E8
indesit,1842777.1299999566
casio,420944.81999999686
baltekstil,20248.070000000014
dauscher,1149407.4599999997


### Step 8 : Benchmark Improvement
I benchmarked performance by comparing query execution time, data scanned, and execution plans before and after optimization. Partitioning enabled partition pruning, and Z-ordering reduced file scans, resulting in faster and more efficient queries.

### Conclusion – Day 10

Day 10 focused on understanding performance optimization in Databricks. By analyzing query execution plans, applying partitioning, and using Z-ORDER optimization, I learned how to reduce data scans and improve query efficiency. Benchmarking before and after optimization helped me understand how these techniques impact performance and scalability in real-world data workloads.